In [2]:
import os
import pandas as pd

In [29]:
tcr_count = pd.read_csv("/Users/fuminli/Desktop/TCR project/data/tcr_count2.csv")
tcr_count = tcr_count.iloc[:,1:4]

In [4]:
hla_I_info = pd.read_csv('/Users/fuminli/Desktop/TCR project/delmonte/HLA_I_info.csv')

In [5]:
folder_path = '/Users/fuminli/Downloads/sampleExport'

file_list = os.listdir(folder_path)

for file_name in file_list:
    
    if file_name.endswith('.tsv'):
        file_path = os.path.join(folder_path, file_name)
        
        var_name = file_name.replace('.tsv', '').replace('-', '_')
        
        globals()[var_name] = pd.read_csv(file_path, sep='\t')

/var/folders/0f/kv13dncx6kb2nbmqhwn99mr00000gn/T/ipykernel_4119/2386243633.py:12: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(file_path, sep='\t')
/var/folders/0f/kv13dncx6kb2nbmqhwn99mr00000gn/T/ipykernel_4119/2386243633.py:12: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(file_path, sep='\t')
/var/folders/0f/kv13dncx6kb2nbmqhwn99mr00000gn/T/ipykernel_4119/2386243633.py:12: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(file_path, sep='\t')
/var/folders/0f/kv13dncx6kb2nbmqhwn99mr00000gn/T/ipykernel_4119/2386243633.py:12: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(file_path, sep='\t')
/var/folders/0f/kv13dncx6kb2nbmqhwn99

In [6]:
immuneACCESS_hla = pd.read_excel("/Users/fuminli/Downloads/1-s2.0-S0091674923025447-mmc2.xlsx")
immuneACCESS_hla.columns = immuneACCESS_hla.iloc[1]
immuneACCESS_hla = immuneACCESS_hla.iloc[2:].reset_index(drop=True)
Participant_id = immuneACCESS_hla.iloc[:,0:2]

In [7]:
tables_dict = {}
for label in Participant_id['HGRepo ID code'].unique():
    tables_dict[label] = pd.DataFrame()

In [8]:
def process_a_column(value):
    if pd.notna(value):
        if '/' in value:
            if '*' in value:
                return value.split('/')[0] + "*" + value.split('*')[1]
            else:
                return value.split('/')[0]
    return value

In [9]:
for index, row in Participant_id.iterrows():
    label = row['HGRepo ID code']
    table_name = row['sample_name']
    table_name = table_name.replace('-', '_')
    current_table = globals().get(table_name)
    
    current_table['v'] = current_table. \
    apply(lambda row: process_a_column(row['vMaxResolved']) if pd.isna(row['vGeneAlleleTies']) else row['vMaxResolved'] + '*' + process_a_column(row['vGeneAlleleTies']).split(',')[0], axis=1)
    
    current_table = current_table.iloc[:,[1,52]]
    aaseq_unique = current_table.iloc[:,[0,1]].drop_duplicates(subset='aminoAcid')
    
    tables_dict[label] = pd.concat([tables_dict[label], aaseq_unique], ignore_index=True)

In [12]:
for index, row in Participant_id.iterrows():
    label = row['HGRepo ID code']
    tables_dict[label] = tables_dict[label].drop_duplicates()

In [11]:
HLA_I_pseudo_40 = pd.read_csv('/Users/fuminli/Downloads/HLA_I_pseudo_40.csv')

In [14]:
hla_I_info.iloc[:, 1:] = 'HLA-' + hla_I_info.iloc[:, 1:].astype(str)

In [15]:
counts = []

for hla_value in HLA_I_pseudo_40['hla']:

    row_presence = hla_I_info.isin([hla_value]).any(axis=1)

    row_count = row_presence.sum()

    counts.append(row_count)

HLA_I_pseudo_40['count'] = counts

In [16]:
HLA_I_pseudo_40_valid = HLA_I_pseudo_40[HLA_I_pseudo_40['count'] != 0]

In [17]:
valid_tcr_values = set(tcr_count['tcr'].unique())

for key in tables_dict.keys():

    tables_dict[key] = tables_dict[key][tables_dict[key]['aminoAcid'].isin(valid_tcr_values)]

In [31]:
hla_table_dict = {}

for hla in HLA_I_pseudo_40_valid['hla'].unique():
    hla_table_dict[hla] = tcr_count.copy()

In [33]:
for key, df in tables_dict.items():
    updated_df = df.drop(df.columns[1], axis=1)  
    updated_df = updated_df.dropna()  
    updated_df = updated_df.drop_duplicates()
    tables_dict[key] = updated_df

In [34]:
merged_tables = {}

for hla, df in hla_table_dict.items():
    relevant_rows = hla_I_info.apply(lambda row: hla in row.values, axis=1)
    relevant_ids = hla_I_info[relevant_rows].iloc[:, 0].unique() 
    list_of_dfs = [tables_dict[table_id] for table_id in relevant_ids if table_id in tables_dict]
    
    if list_of_dfs:
        merged_df = pd.concat(list_of_dfs)
        merged_df = merged_df.drop_duplicates(subset='aminoAcid')
        merged_df['cooccur'] = merged_df['aminoAcid'].map(pd.concat(list_of_dfs)['aminoAcid'].value_counts())
        merged_tables[hla] = merged_df
    else:
        merged_tables[hla] = pd.DataFrame(columns=df.columns)


In [37]:
list_of_dfs = list(tables_dict.values()) 
merged_df = pd.concat(list_of_dfs)  
merged_df['count'] = merged_df.groupby('aminoAcid').cumcount() + 1

final_df = merged_df.drop_duplicates(list(merged_df.columns[:-1]), keep='first')

In [39]:
final_df = final_df.reset_index().iloc[:,1:4]
idx = final_df.groupby('aminoAcid')['count'].idxmax()  
max_v_df = final_df.loc[idx]

In [31]:
max_v_df = max_v_df.reset_index().iloc[:,1:3]
tcr_count = tcr_count.dropna().merge(max_v_df, left_on='tcr', right_on='aminoAcid', how='left').iloc[:,[0,3,1]]

In [33]:
hla_final_dict = {}

for hla, df in hla_table_dict.items():
    hla_final_dict[hla] = tcr_count.merge(merged_tables[hla], left_on="tcr", right_on="aminoAcid").iloc[:,[0,1,2,4]]

In [ ]:
folder_path = '/Users/fuminli/Desktop/TCR project/delmonte/HLA_I_cooccur'
for key, df in hla_final_dict.items():
    file_path = os.path.join(folder_path, f'{key}.csv')
    df.to_csv(file_path, index=False)

In [ ]:
HLA_I_pseudo_40_valid.to_csv('/Users/fuminli/Desktop/TCR project/delmonte/HLA_I_count.csv')